# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [14]:
# Load the dotenv extension
%load_ext dotenv

# Load environment variables from .env
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [15]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [16]:
import os
from glob import glob

# Load the environment variable PRICE_DATA
ft_dir = os.getenv('PRICE_DATA')

# If recursive is true, the pattern “**” will match any files and zero or more directories, subdirectories and symbolic links to directories.
parquet_files = glob(ft_dir + '**/*.parquet', recursive=True)

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [17]:
# Reading multiple parquet files into a Dask DataFrame
df = dd.read_parquet(parquet_files)

# Grouping by Ticker column and group_keys = False ensures that the group labels do not appear in the index of the result. 
# .apply() will apply the given function to each group. 
# lambda is the other way of defining a function. E.g .shift(1) will take the Close column value from the previous row and assign it to the new column name Close_lag_1.

df_lags = df.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1),
                       Close_lag_2 = x['Adj Close'].shift(1))
)

dd_feat = df_lags.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(returns = (x['Close'] / x['Close_lag_1']) - 1,
                       hi_lo_range = x['High'] - x['Low'])
)

C:\Users\15199\AppData\Local\Temp\ipykernel_2180\3633471738.py:8: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  df_lags = df.groupby('Ticker', group_keys=False).apply(
C:\Users\15199\AppData\Local\Temp\ipykernel_2180\3633471738.py:13: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = df_lags.groupby('Ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [18]:
# Converting Dask data frame to a pandas data frame. Further calculating moving average using a window of 10 days. First nine values will be NaN as there are not 10 values for the mean.
dd_feat = dd_feat.compute()
dd_feat['rolling_mean'] = dd_feat['returns'].rolling(10).mean()
dd_feat

,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Close_lag_2,returns,hi_lo_range,rolling_mean
322016,2020-01-02 00:00:00+00:00,JNPR,21.375463,24.709999,24.900000,24.540001,24.809999,2876000.0,2020,NaN,NaN,NaN,0.359999,NaN
322080,2020-01-03 00:00:00+00:00,JNPR,21.055395,24.340000,24.510000,24.170000,24.260000,3041800.0,2020,24.709999,21.375463,-0.014974,0.340000,NaN
322144,2020-01-06 00:00:00+00:00,JNPR,21.012142,24.290001,24.410000,24.139999,24.160000,4141100.0,2020,24.340000,21.055395,-0.002054,0.270000,NaN
322208,2020-01-07 00:00:00+00:00,JNPR,20.925638,24.190001,24.400000,24.059999,24.400000,1909500.0,2020,24.290001,21.012142,-0.004117,0.340000,NaN
322272,2020-01-08 00:00:00+00:00,JNPR,20.856436,24.110001,24.320000,24.020000,24.129999,2143000.0,2020,24.190001,20.925638,-0.003307,0.299999,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225066,2013-12-24 00:00:00+00:00,NTAP,30.642590,40.380001,40.540001,40.279999,40.320000,1799900.0,2013,40.380001,30.642590,0.000000,0.260002,-0.002559
225130,2013-12-26 00:00:00+00:00,NTAP,30.589462,40.310001,40.660000,40.220001,40.220001,1862500.0,2013,40.380001,30.642590,-0.001734,0.439999,0.000645
225194,2013-12-27 00:00:00+00:00,NTAP,30.885426,40.700001,40.810001,40.320000,40.330002,1943600.0,2013,40.310001,30.589462,0.009675,0.490002,0.002760
225258,2013-12-30 00:00:00+00:00,NTAP,31.097897,40.980000,41.090000,40.709999,40.730000,1918500.0,2013,40.700001,30.885426,0.006880,0.380001,0.003019


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

+ It's not necessary to convert to pandas to calculate the moving average return. We can also work with the csv files. Also, repartition() can be used with map_overlap() without converting Dask DataFrame to pandas for calculating the rolling average.  
Dask DataFrame is lazy, meaning it doesn't compute results immediately. It does not show actual data; it just provides an overview. So, the other method is to use .compute() to show actual results to calculate the rolling average, and the compute method converts the dask DataFrame to pandas DataFrame. For data that fits into RAM, pandas can often be faster and easier to use than Dask DataFrame.  

+ It depends on the size of the dataset. If it's small, pandas is the better than Dask. However, if it's large dataset, Dask is the best choice. Our dataset is not very large, so converting to pandas is good choice.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.